In [35]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, streaming=True,callbacks=[StreamingStdOutCallbackHandler()])

poetry_prgramming_language_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international poet. You create easy to understand for any type of poem with a programming language."),
    ("human", "I want to write a poetry about {programming_language}.")
])

poetry_prgramming_language_chain = poetry_prgramming_language_prompt | chat

In [36]:
explain_poetry_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a poetry explainer: do not state a single definitive meaning—first cite textual evidence (lines, structure, repetition, imagery), then propose interpretations. If you discuss background or authorial intent without solid support, label it as “uncertain,” and offer 2 or 3 plausible readings, choosing the most convincing one with clear reasons. Default output order: one-line thesis → plain-language paraphrase → analysis of imagery, devices, rhythm, and themes → reader questions and re-reading checkpoints."), ("human", "{poetry}")
])

explain_poetry_chain = explain_poetry_prompt | chat

final_chain = {"poetry": poetry_prgramming_language_chain} | explain_poetry_chain

final_chain.invoke({
    "programming_language": "python"
})

In the land of code, where serpents slither,
Python reigns, a language that's neither
Too complex nor too verbose,
Its syntax flows like a gentle prose.

Indentation matters, in Python's domain,
No curly braces to cause you pain.
With libraries galore, and functions to call,
Python stands tall, above them all.

From data science to web design,
Python's versatility will surely shine.
So embrace the snake, let your creativity soar,
In the world of Python, there's always more.The poem "In the Land of Code" celebrates the programming language Python for its simplicity, elegance, and versatility, likening it to a gentle and powerful force in the world of coding.

The poem uses imagery of serpents, a reference to Python's namesake, to convey the language's presence and influence in the coding world. The comparison of Python to a serpent suggests both its sleekness and its potential to be a powerful tool.

The repetition of the word "Python" throughout the poem emphasizes the central theme of

AIMessage(content='The poem "In the Land of Code" celebrates the programming language Python for its simplicity, elegance, and versatility, likening it to a gentle and powerful force in the world of coding.\n\nThe poem uses imagery of serpents, a reference to Python\'s namesake, to convey the language\'s presence and influence in the coding world. The comparison of Python to a serpent suggests both its sleekness and its potential to be a powerful tool.\n\nThe repetition of the word "Python" throughout the poem emphasizes the central theme of the language\'s importance and dominance in the realm of coding. This repetition serves to reinforce Python\'s significance and versatility.\n\nThe poem\'s structure, with its rhyming couplets and consistent meter, creates a sense of rhythm and flow that mirrors Python\'s smooth syntax and ease of use. This structure reflects the poem\'s message about Python\'s simplicity and elegance.\n\nThemes of creativity, exploration, and endless possibilities